# Trabajo Práctico 2 - Clasificación aplicada a la EPH 

## Gil Deza, Hüppi Lo Prete, Walker 

### Parte 1 - Analizando la base 

In [2]:
# Importamos paquetes a utilizar
import pandas as pd

### Punto 2

In [26]:
# Importamos la base
df=pd.read_excel("usu_individual_T122.xls")
df


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,2,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
1,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,3,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
2,TQRMNOQXQHLOKQCDEGKDB00777573,2022,1,1,4,1,43,N,14,104,...,10.0,10,77500.0,9,9.0,9,NaN,9.0,10,194
3,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,1,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
4,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,2,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49701,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,2,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49702,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,1,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777
49703,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,2,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777
49704,TQRMNOQRPHMMLPCDEFIAH00707721,2022,1,1,1,1,43,S,2,678,...,NaN,6,34000.0,5,5.0,5,5.0,NaN,5,2274


In [37]:
# Armamos un nuevo df solo con las observaciones de Buenos Aires y Gran Buenos Aires
df_ba = df.loc[df['AGLOMERADO'].isin([32, 33])]
df_ba


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
3,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,1,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
4,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,2,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
5,TQRMNOSUPHKKPQCDEIJAH00780151,2022,1,1,3,1,1,S,33,1741,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
75,TQRMNOPQQHKMRLCDEIJAH00780169,2022,1,1,1,1,1,S,33,456,...,NaN,0,0.0,0,NaN,0,0.0,NaN,0,647
76,TQRMNOSXRHJMTRCDEIJAH00693084,2022,1,1,1,1,1,S,33,1199,...,NaN,1,10000.0,1,NaN,1,1.0,NaN,1,1470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49699,TQRMNOTTSHKNLSCDEIIAD00780102,2022,1,1,2,1,1,S,32,2839,...,NaN,12,0.0,12,NaN,12,12.0,NaN,12,0
49700,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,1,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49701,TQRMNOPRPHMNMLCDEIIAD00701192,2022,1,1,2,1,1,S,32,2131,...,NaN,9,187500.0,10,NaN,10,10.0,NaN,10,5143
49702,TQRMNOSRWHKMLUCDEIIAD00780103,2022,1,1,1,1,1,S,32,1513,...,NaN,5,56100.0,8,NaN,8,8.0,NaN,5,2777


In [50]:
df_ba[negativos] = df_ba["ITF"] < 0
df_ba[negativos].values_count()

C:\Users\caterina\AppData\Local\Temp\ipykernel_3064\81328755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ba[negativos] = df_ba["ITF"] < 0


AttributeError: 'DataFrame' object has no attribute 'values_count'